## import previous file

In [1]:
import pandas as pd
from pathlib import Path

p = Path("/Users/dengshuyue/Desktop/SDOH/analysis/output/cov_addv4_99_23.parquet")
df_my_cov_aligned_short = pd.read_parquet(p)  # uses pyarrow/fastparquet if available
print(df_my_cov_aligned_short.shape)
df_my_cov_aligned_short.head()


(128809, 68)


,SEQN,SDDSRVYR,sdmvpsu,sdmvstra,RIDAGEYR,SEX,RACE,re,household_size,DMDHHSIZ,...,HOQ065_structural_missing,household_size_structural_missing,chol_rx_structural_missing,RIAGENDR,drinking,alcg2,perE_alco,METSCORE_fromPAQ,LTPA_fromPAQ,met_hr_recalc_from
0,1,1.0,1,5,2.0,F,4.0,Other Hispanic,3,3.0,...,False,False,False,2.0,NaN,<NA>,0.000000,NaN,NaN,None
1,2,1.0,3,1,77.0,M,3.0,Mexican American,1,1.0,...,False,False,False,1.0,0.065753,2,0.000000,NaN,NaN,from_new_PAQ
2,3,1.0,2,7,10.0,F,3.0,Mexican American,4,4.0,...,False,False,False,2.0,NaN,<NA>,0.000000,NaN,NaN,None
3,4,1.0,1,2,1.0,M,4.0,Other Hispanic,7,7.0,...,False,False,False,1.0,NaN,<NA>,0.000000,NaN,NaN,None
4,5,1.0,2,8,49.0,M,3.0,Mexican American,3,3.0,...,False,False,False,1.0,1.714286,2,9.101101,NaN,NaN,from_new_PAQ


## show missing 

In [2]:
df = df_my_cov_aligned_short

# use every column except the grouper
# ignore PACK_YEARS, CIGS_PER_DAY, probable_depression and etc as missing naturally 
# ignore dulicate safe saved old column name

exclude = {"SDDSRVYR","CIGS_PER_DAY","PACK_YEARS","probable_depression","wt_phlebotomy", "WTSAFPRP",
           "WTINT2YR", "WTMEC2YR", "WTPH2YR", "WTSAF2YR", "WTMEC4YR", "WTINTPRP", "WTMECPRP", "WTINT4YR",
           "SNAP", "SNAP_src", "SNAP_bin", "SNAP_src_rank", "bmi", "RIAGENDR",
           "SNAP_indiv_only","FS", "ahei_total", "HOQ065", "marriage_label", "marriage_prev",
           "METSCORE_fromPAQ","perE_alco","LTPA_fromPAQ","SNAP_indiv_plus_singleton", 
           "SMK_AVG", "BMI_CLAS", "household_size", "DMDHHSIZ"}
cols_all = [c for c in df.columns if c not in exclude]

# % missing by cycle (split into two lines)
is_na = df[cols_all].isna()
pct_miss = is_na.groupby(df["SDDSRVYR"]).mean().mul(100)

# keep only columns that exceed 80% missing in ANY cycle
pct_miss_gt80 = pct_miss.loc[:, (pct_miss > 80).any(axis=0)].round(1)

print(pct_miss_gt80)


          chol_rx  alcg2
SDDSRVYR                
1.0           0.0   62.9
2.0           0.0   66.5
3.0           0.0   62.9
4.0           0.0   62.7
5.0           0.0   55.4
6.0           0.0   55.4
7.0           0.0   55.1
8.0           0.0   52.2
9.0           0.0   52.8
10.0          0.0   56.0
12.0         29.1   61.4
66.0        100.0   84.3


## fetch chol and code chol_rx

In [35]:
import io, re, requests
import pandas as pd
from pandas.api.types import is_numeric_dtype

# =========================
# Core helpers
# =========================
def _read_xpt(url, cols_upper=True):
    r = requests.get(url); r.raise_for_status()
    df = pd.read_sas(io.BytesIO(r.content), format="xport", encoding="latin1")
    if cols_upper:
        df.columns = [c.upper() for c in df.columns]
    if "SEQN" in df.columns:
        df["SEQN"] = pd.to_numeric(df["SEQN"], errors="coerce").astype("Int64")
    return df

def _try_read_xpt(url, cols_upper=True):
    """Return empty DataFrame on HTTP errors so optional files don't crash."""
    try:
        if not url:
            return pd.DataFrame()
        return _read_xpt(url, cols_upper=cols_upper)
    except requests.HTTPError:
        return pd.DataFrame()

def _find_numeric_col(df, preferred, fallback_regex):
    """Pick first existing preferred col, else first numeric matching fallback regex."""
    if df is None or df.empty:
        return None
    for c in preferred:
        if c in df.columns and is_numeric_dtype(df[c]):
            return c
    pats = re.compile(fallback_regex, flags=re.I)
    for c in df.columns:
        if pats.search(c) and is_numeric_dtype(df[c]):
            return c
    return None

def _friedewald_ldl(tc, hdl, tg):
    """LDL = TC - HDL - TG/5 when TG < 400; else NA."""
    ldl = pd.Series(pd.NA, index=tc.index, dtype="Float64")
    cond = tc.notna() & hdl.notna() & tg.notna() & (tg < 400)
    ldl.loc[cond] = (tc.loc[cond] - hdl.loc[cond] - (tg.loc[cond] / 5.0))
    return ldl

# =========================
# Period-specific sources
# =========================
BASE_2017 = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles"
BASE_2021 = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles"

URLS_P = {
    "bpq":   f"{BASE_2017}/P_BPQ.xpt",      # BPQ080 (dx), BPQ090D (also used), BPQ100D (meds; not used for flag)
    "hdl":   f"{BASE_2017}/P_HDL.xpt",      # HDL
    "tchol": f"{BASE_2017}/P_TCHOL.xpt",    # Total chol (and sometimes direct LDL)
    "trig":  f"{BASE_2017}/P_TRIGLY.xpt",   # TG (optional)
}

URLS_L = {
    "bpq":   f"{BASE_2021}/BPQ_L.xpt",      # BPQ080, BPQ090D (if present), BPQ101D
    "hdl":   f"{BASE_2021}/HDL_L.xpt",
    "tchol": f"{BASE_2021}/TCHOL_L.xpt",
    # TRIGLY_L not published
}

# =========================
# Builder for a given period (matches SAS: high_chol2 = high_chol OR (LBXTC > 200))
# =========================
def build_period_df(urls: dict, period_label: str) -> pd.DataFrame:
    # --- BPQ: ever told high cholesterol ---
    bpq = _read_xpt(urls["bpq"])
    bpq_keep = bpq[["SEQN"]].copy()

    # DX = (BPQ080 == 1) OR (BPQ090D == 1 if present)
    has_080 = pd.to_numeric(bpq.get("BPQ080"), errors="coerce").eq(1) if "BPQ080" in bpq.columns else pd.Series(False, index=bpq.index)
    has_090 = pd.to_numeric(bpq.get("BPQ090D"), errors="coerce").eq(1) if "BPQ090D" in bpq.columns else pd.Series(False, index=bpq.index)
    bpq_keep["hc_dx"] = (has_080 | has_090).astype("Int64")

    # --- Labs (for flag and summaries) ---
    hdl  = _read_xpt(urls["hdl"])
    tch  = _read_xpt(urls["tchol"])
    trig = _try_read_xpt(urls.get("trig", ""), cols_upper=True) if "trig" in urls else pd.DataFrame()

    # Robust variable discovery
    hdl_var = _find_numeric_col(hdl, preferred=["LBDHDD", "LBXHDD", "LBDHDD_1"], fallback_regex=r"\bHDL\b|HDDS?\b|^LB..HD")
    tc_var  = _find_numeric_col(tch, preferred=["LBXTC"], fallback_regex=r"\bTOTAL.*CHOL|^LB..TC$|\bTC\b")
    ldl_var = _find_numeric_col(tch, preferred=["LBDLDL", "LBDLDLD", "LBDLDLL"], fallback_regex=r"\bLDL\b|^LB.DLDL")
    tg_var  = _find_numeric_col(trig, preferred=["LBXTR"], fallback_regex=r"\bTRI?G|^LB..TR$") if not trig.empty else None

    # Keep frames
    hdl_keep = hdl[["SEQN", hdl_var]].rename(columns={hdl_var: "hdl_mgdl"}) if hdl_var else pd.DataFrame(columns=["SEQN","hdl_mgdl"])
    tc_keep  = tch[["SEQN", tc_var ]].rename(columns={tc_var : "total_chol_mgdl"}) if tc_var  else pd.DataFrame(columns=["SEQN","total_chol_mgdl"])
    ldl_keep = tch[["SEQN", ldl_var]].rename(columns={ldl_var: "ldl_mgdl"}) if ldl_var else pd.DataFrame(columns=["SEQN","ldl_mgdl"])
    tg_keep  = trig[["SEQN", tg_var]].rename(columns={tg_var: "trig"}) if tg_var else pd.DataFrame(columns=["SEQN","trig"])

    # Merge
    df = (
        bpq_keep
        .merge(hdl_keep, how="left", on="SEQN")
        .merge(tc_keep,  how="left", on="SEQN")
        .merge(ldl_keep, how="left", on="SEQN")
        .merge(tg_keep,  how="left", on="SEQN")
    )

    # Compute LDL if missing and feasible (for summaries only; not used in flag)
    need_calc = df["ldl_mgdl"].isna() if "ldl_mgdl" in df.columns else pd.Series(False, index=df.index)
    have_cols = {"total_chol_mgdl","hdl_mgdl","trig"} <= set(df.columns)
    if need_calc.any() and have_cols and df["trig"].notna().any():
        ldl_est = _friedewald_ldl(df["total_chol_mgdl"], df["hdl_mgdl"], df["trig"])
        df.loc[need_calc, "ldl_mgdl"] = ldl_est.loc[need_calc]

    # SAS lab rule: LBXTC > 200 (strictly greater than 200)
    df["hc_lab200"] = (df["total_chol_mgdl"] > 200).astype("Int64")

    # Final SAS-aligned flag: high_chol2 = high_chol (dx) OR (LBXTC > 200)
    df["chol_rx"] = ((df["hc_dx"].fillna(0).astype(int) == 1) | (df["hc_lab200"].fillna(0).astype(int) == 1)).astype(int)

    df["period"] = period_label

    # Normalize numeric dtypes
    for c in ["total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").astype("Float64")

    keep_cols = ["SEQN","period","chol_rx","hc_dx","hc_lab200",
                 "total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"]
    for c in keep_cols:
        if c not in df.columns:
            df[c] = pd.NA
    return df[keep_cols]

# =========================
# Build both periods & QC (SAS-aligned definition)
# =========================
df_p = build_period_df(URLS_P, period_label="2017-2020 (P)")
df_l = build_period_df(URLS_L, period_label="2021-2022 (L)")
df_both = pd.concat([df_p, df_l], ignore_index=True)

qc = (
    df_both.groupby("period", dropna=False)
    .agg(n=("SEQN","count"),
         chol_rx_rate=("chol_rx","mean"),
         dx_rate=("hc_dx","mean"),
         lab200_rate=("hc_lab200","mean"),
         mean_tc=("total_chol_mgdl","mean"),
         mean_hdl=("hdl_mgdl","mean"),
         mean_ldl=("ldl_mgdl","mean"))
    .round(3)
)
print(qc)

# =========================
# (Optional) mmol/L conversions
# =========================
# df_both["tc_mmol"]  = (df_both["total_chol_mgdl"] * 0.02586).astype("Float64")
# df_both["hdl_mmol"] = (df_both["hdl_mgdl"]        * 0.02586).astype("Float64")
# df_both["ldl_mmol"] = (df_both["ldl_mgdl"]        * 0.02586).astype("Float64")

# =========================
# (Optional) Join into your main covariate DF and save
# =========================
# df_my_cov_aligned_short = df_my_cov_aligned_short.merge(
#     df_both.drop(columns=["period"]), on="SEQN", how="left"
# )
# out_path = "cov_addv4_99_23.parquet"
# df_my_cov_aligned_short.to_parquet(out_path, index=False)
# print("✓ Saved:", out_path)


                   n  chol_rx_rate  dx_rate  lab200_rate  mean_tc  mean_hdl  \
period                                                                        
2017-2020 (P)  10195         0.502    0.362        0.259  183.056    53.339   
2021-2022 (L)   8501         0.493    0.364        0.233  185.691    54.314   

               mean_ldl  
period                   
2017-2020 (P)   107.112  
2021-2022 (L)      <NA>  



--- Marginal & overlap rates ---
                   n  dx_rate  lab200_rate  both_rate  union_rate
period                                                           
2017-2020 (P)  10195    0.362        0.259      0.119       0.502
2021-2022 (L)   8501    0.364        0.233      0.104       0.493

--- Combination breakdown (counts, share of sample, share among chol_rx=1) ---
          period combo_label     n  share_of_sample  share_among_chol_rx
0  2017-2020 (P)        none  5082            0.498                0.000
1  2017-2020 (P)    lab only  1426            0.140                0.279
2  2017-2020 (P)     dx only  2473            0.243                0.484
3  2017-2020 (P)      dx+lab  1214            0.119                0.237
4  2021-2022 (L)        none  4313            0.507                0.000
5  2021-2022 (L)    lab only  1092            0.128                0.261
6  2021-2022 (L)     dx only  2211            0.260                0.528
7  2021-2022 (L)      dx+lab   885    

#### check why chol_rx rate is high

In [32]:
import pandas as pd

df = df_both.copy()
for c in ["hc_dx","hc_lab200","chol_rx"]:
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

# Sanity: chol_rx should be the union of dx and lab200
if not (df["chol_rx"].eq((df["hc_dx"] | df["hc_lab200"]).astype(int))).all():
    raise ValueError("chol_rx is not equal to (hc_dx OR hc_lab200) somewhere.")

# 1) Marginal and overlap rates (per period)
df["both"] = (df["hc_dx"].eq(1) & df["hc_lab200"].eq(1)).astype(int)
margins = (
    df.groupby("period", dropna=False)
      .agg(n=("SEQN","count"),
           dx_rate=("hc_dx","mean"),
           lab200_rate=("hc_lab200","mean"),
           both_rate=("both","mean"),
           union_rate=("chol_rx","mean"))
      .round(3)
)
print("\n--- Marginal & overlap rates ---")
print(margins)

# 2) Combination counts & shares
# combos: 00=none, 10=dx only, 01=lab only, 11=both
df["combo"] = df[["hc_dx","hc_lab200"]].astype(str).agg("".join, axis=1)
combo_map = {"00":"none","10":"dx only","01":"lab only","11":"dx+lab"}

combo_counts = (
    df.groupby(["period","combo"], as_index=False)
      .size()
      .rename(columns={"size":"n"})
)
# share of full sample
combo_counts["share_of_sample"] = (
    combo_counts["n"] / combo_counts.groupby("period")["n"].transform("sum")
).round(3)

# share among chol_rx = 1 (exclude 'none' by definition)
n_pos = df.groupby("period")["chol_rx"].sum().rename("n_pos")
combo_counts = combo_counts.merge(n_pos, on="period", how="left")
combo_counts["share_among_chol_rx"] = (
    combo_counts.apply(lambda r: r["n"]/r["n_pos"] if (r["combo"]!="00" and r["n_pos"]>0) else 0, axis=1)
).round(3)

combo_counts["combo_label"] = combo_counts["combo"].map(combo_map)
combo_counts = combo_counts.sort_values(["period","combo"])

print("\n--- Combination breakdown (counts, share of sample, share among chol_rx=1) ---")
print(combo_counts[["period","combo_label","n","share_of_sample","share_among_chol_rx"]])

# 3) Quick “drivers among positives”
drivers = (
    df.loc[df["chol_rx"]==1]
      .groupby("period")[["hc_dx","hc_lab200","both"]]
      .mean()
      .rename(columns={
          "hc_dx":"share_pos_with_dx",
          "hc_lab200":"share_pos_with_lab200",
          "both":"share_pos_with_both"
      })
      .round(3)
)
print("\n--- Among chol_rx=1: shares with dx, lab200, and both present ---")
print(drivers)



--- Marginal & overlap rates ---
                   n  dx_rate  lab200_rate  both_rate  union_rate
period                                                           
2017-2020 (P)  10195    0.362        0.259      0.119       0.502
2021-2022 (L)   8501    0.364        0.233      0.104       0.493

--- Combination breakdown (counts, share of sample, share among chol_rx=1) ---
          period combo_label     n  share_of_sample  share_among_chol_rx
0  2017-2020 (P)        none  5082            0.498                0.000
1  2017-2020 (P)    lab only  1426            0.140                0.279
2  2017-2020 (P)     dx only  2473            0.243                0.484
3  2017-2020 (P)      dx+lab  1214            0.119                0.237
4  2021-2022 (L)        none  4313            0.507                0.000
5  2021-2022 (L)    lab only  1092            0.128                0.261
6  2021-2022 (L)     dx only  2211            0.260                0.528
7  2021-2022 (L)      dx+lab   885    

In [33]:
p = df_both.query("period == '2017-2020 (P)'").copy()
# overall (marginal) dx rate
dx_rate = (p["hc_dx"] == 1).mean()

# conditional share among chol_rx positives
share_dx_among_pos = (p.loc[p["chol_rx"] == 1, "hc_dx"] == 1).mean()

# counts to match your tables
n = p.shape[0]
n_dx = (p["hc_dx"] == 1).sum()
n_pos = (p["chol_rx"] == 1).sum()
n_dx_among_pos = (p["hc_dx"].eq(1) & p["chol_rx"].eq(1)).sum()

print("n, n_dx, n_pos, n_dx_among_pos =", n, n_dx, n_pos, n_dx_among_pos)
print("dx_rate (overall) =", round(dx_rate, 3))
print("share_dx_among_chol_rx =", round(share_dx_among_pos, 3))


n, n_dx, n_pos, n_dx_among_pos = 10195 3687 5113 3687
dx_rate (overall) = 0.362
share_dx_among_chol_rx = 0.721


#### check if previous cov code have mention chol_rx method

In [26]:
from pathlib import Path
import re

"""
p = Path("/Users/dengshuyue/Desktop/SDOH/analysis/2.1_Prepare data_covariates.sas")
pat = re.compile(r'chol(_rx)?|HIGH_CHOL|BPQ0?80|BPQ10(0D|1D)|LBXTC|LBDLDL|LBXTR|HDL|RXQ_RX|RXDDRUG|STATIN|EZETIMIBE|FIBRATE', re.I)

lines = p.read_text(errors="ignore").splitlines()
for i, line in enumerate(lines):
    if pat.search(line):
        start = max(0, i-3); end = min(len(lines), i+4)
        print(f"\n--- lines {start+1}-{end} ---")
        for j in range(start, end):
            print(f"{j+1:5d}: {lines[j]}")
"""


'\np = Path("/Users/dengshuyue/Desktop/SDOH/analysis/2.1_Prepare data_covariates.sas")\npat = re.compile(r\'chol(_rx)?|HIGH_CHOL|BPQ0?80|BPQ10(0D|1D)|LBXTC|LBDLDL|LBXTR|HDL|RXQ_RX|RXDDRUG|STATIN|EZETIMIBE|FIBRATE\', re.I)\n\nlines = p.read_text(errors="ignore").splitlines()\nfor i, line in enumerate(lines):\n    if pat.search(line):\n        start = max(0, i-3); end = min(len(lines), i+4)\n        print(f"\n--- lines {start+1}-{end} ---")\n        for j in range(start, end):\n            print(f"{j+1:5d}: {lines[j]}")\n'

In [7]:
df = df_my_cov_aligned_short.copy()

# --- make sure chol_rx is a clean 0/1 Int64 and ignore NAs for the rate denom ---
if "chol_rx" not in df.columns:
    raise ValueError("chol_rx not found. Merge the cholesterol builder outputs first.")
chol = pd.to_numeric(df["chol_rx"], errors="coerce").astype("Int64")

# =============== Unweighted prevalence by cycle ===============
unw = (
    pd.DataFrame({"SDDSRVYR": df["SDDSRVYR"], "chol_rx": chol})
    .dropna(subset=["chol_rx"])
    .groupby("SDDSRVYR")
    .agg(n=("chol_rx", "size"),
         cases=("chol_rx", "sum"))
    .assign(chol_rx_rate=lambda d: d["cases"]/d["n"])
    .reset_index()
)

print("Unweighted chol_rx prevalence by cycle:")
print(unw)

# =============== Weighted prevalence by cycle (optional) ===============
# Prefer WTMEC2YR (exam weights). If not present, fall back to WTINT2YR.
wt_col = None
for cand in ["WTMEC2YR", "WTINT2YR", "WTMECPRP", "WTINTPRP"]:
    if cand in df.columns:
        wt_col = cand
        break

if wt_col:
    tmp = (
        pd.DataFrame({
            "SDDSRVYR": df["SDDSRVYR"],
            "chol_rx": chol,
            "wt": pd.to_numeric(df[wt_col], errors="coerce")
        })
        .dropna(subset=["chol_rx", "wt"])
    )
    w = (
        tmp.assign(w_cases=lambda d: d["wt"]*d["chol_rx"])
          .groupby("SDDSRVYR")
          .agg(w_sum=("wt","sum"),
               w_cases=("w_cases","sum"),
               n=("chol_rx","size"))  # n shown for reference
          .assign(chol_rx_rate_wt=lambda d: d["w_cases"]/d["w_sum"])
          .reset_index()
    )
    print(f"\n{wt_col}-weighted chol_rx prevalence by cycle:")
    print(w[["SDDSRVYR","n","chol_rx_rate_wt"]].rename(columns={"chol_rx_rate_wt":"chol_rx_rate"}))
else:
    print("\nNo weight column found; skipped weighted prevalence.")


Unweighted chol_rx prevalence by cycle:
    SDDSRVYR      n  cases  chol_rx_rate
0        1.0   9965    422      0.042348
1        2.0  11039    613       0.05553
2        3.0  10122    735      0.072614
3        4.0  10348    773        0.0747
4        5.0  10149   1194      0.117647
5        6.0  10537   1232      0.116921
6        7.0   9756   1100      0.112751
7        8.0  10175   1226      0.120491
8        9.0   9971   1170       0.11734
9       10.0   9254   1318      0.142425
10      12.0   8465   2126      0.251152

WTMEC2YR-weighted chol_rx prevalence by cycle:
    SDDSRVYR      n  chol_rx_rate
0        1.0   9282      0.055405
1        2.0  10477      0.070017
2        3.0   9643      0.090047
3        4.0   9950      0.103745
4        5.0   9762      0.119686
5        6.0  10253      0.125808
6        7.0   9338      0.135788
7        8.0   9813      0.152367
8        9.0   9544      0.138229
9       10.0   8704      0.145363
10      12.0   8465      0.211396


#### Full cycle fetch 99-23

In [37]:
import io, re, requests
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype

# =========================
# Helpers
# =========================
def _read_xpt(url, cols_upper=True):
    r = requests.get(url); r.raise_for_status()
    df = pd.read_sas(io.BytesIO(r.content), format="xport", encoding="latin1")
    if cols_upper:
        df.columns = [c.upper() for c in df.columns]
    if "SEQN" in df.columns:
        df["SEQN"] = pd.to_numeric(df["SEQN"], errors="coerce").astype("Int64")
    return df

def _try_read_xpt(url, cols_upper=True):
    try:
        if not url:
            return pd.DataFrame()
        return _read_xpt(url, cols_upper=cols_upper)
    except requests.HTTPError:
        return pd.DataFrame()

def _find_numeric_col(df, preferred, fallback_regex):
    if df is None or df.empty:
        return None
    for c in preferred:
        if c in df.columns and is_numeric_dtype(df[c]):
            return c
    pats = re.compile(fallback_regex, flags=re.I)
    for c in df.columns:
        if pats.search(c) and is_numeric_dtype(df[c]):
            return c
    return None

def _friedewald_ldl(tc, hdl, tg):
    """LDL = TC - HDL - TG/5 when TG < 400; else NA."""
    ldl = pd.Series(pd.NA, index=tc.index, dtype="Float64")
    cond = tc.notna() & hdl.notna() & tg.notna() & (tg < 400)
    ldl.loc[cond] = (tc.loc[cond] - hdl.loc[cond] - (tg.loc[cond] / 5.0))
    return ldl

def wmean(x, w):
    x = pd.to_numeric(x, errors="coerce")
    w = pd.to_numeric(w, errors="coerce")
    m = x.notna() & w.notna() & (w > 0)
    if m.sum() == 0:
        return np.nan
    return float((x[m] * w[m]).sum() / w[m].sum())

# =========================
# Cycle plumbing (1999–2018)
# =========================
CYCLES = [
    ("",   "1999-2000"),
    ("_B", "2001-2002"),
    ("_C", "2003-2004"),
    ("_D", "2005-2006"),
    ("_E", "2007-2008"),
    ("_F", "2009-2010"),
    ("_G", "2011-2012"),
    ("_H", "2013-2014"),
    ("_I", "2015-2016"),
    ("_J", "2017-2018"),
]

def _base_folder(suffix):
    return {
        "":   "1999", "_B": "2001", "_C": "2003", "_D": "2005", "_E": "2007",
        "_F": "2009", "_G": "2011", "_H": "2013", "_I": "2015", "_J": "2017"
    }[suffix]

def _url(folder, filebase):
    return f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/{folder}/DataFiles/{filebase}.xpt"

def _tc_hdl_candidates(suffix):
    """Return ordered candidates for Total/HDL file names by cycle."""
    if suffix == "":      # 1999–2000
        return ["LAB13", "TCHOL", "HDL"]  # LAB13 preferred; fallbacks split
    s = suffix.lstrip("_")
    if suffix in {"_B","_C"}:
        return [f"L13_{s}", f"TCHOL{suffix}", f"HDL{suffix}"]
    # 2005+ use split files
    return [f"TCHOL{suffix}", f"HDL{suffix}"]

def _ldl_tg_candidates(suffix):
    """Return ordered candidates for LDL/TG file names by cycle."""
    if suffix == "":      # 1999–2000
        return ["LAB13AM", "TRIGLY", "TCHOL"]  # LAB13AM preferred; fallbacks
    s = suffix.lstrip("_")
    if suffix in {"_B","_C"}:
        return [f"L13AM_{s}", f"TRIGLY{suffix}", f"TCHOL{suffix}"]
    # 2005+ use split files
    return [f"TRIGLY{suffix}", f"TCHOL{suffix}"]

# =========================
# Build ONE cycle (SAS-aligned: dx = 080 OR 090D; flag = dx OR (TC>200))
# =========================
def build_cycle_df(suffix: str, label: str) -> pd.DataFrame:
    folder = _base_folder(suffix)

    # --- BPQ (dx items) ---
    bpq = _try_read_xpt(_url(folder, f"BPQ{suffix}"))
    bpq_keep = pd.DataFrame()
    if not bpq.empty:
        bpq_keep = bpq[["SEQN"]].copy()
        has_080 = pd.to_numeric(bpq.get("BPQ080"),  errors="coerce").eq(1) if "BPQ080"  in bpq.columns else pd.Series(False, index=bpq.index)
        has_090 = pd.to_numeric(bpq.get("BPQ090D"), errors="coerce").eq(1) if "BPQ090D" in bpq.columns else pd.Series(False, index=bpq.index)
        bpq_keep["hc_dx"] = (has_080 | has_090).astype("Int64")

    # --- Labs: try multiple filename patterns per cycle ---
    # Total & HDL
    tc_hdl = pd.DataFrame()
    for base in _tc_hdl_candidates(suffix):
        tc_hdl = _try_read_xpt(_url(folder, base))
        if not tc_hdl.empty:
            break

    # LDL & TG
    ldl_tg = pd.DataFrame()
    for base in _ldl_tg_candidates(suffix):
        ldl_tg = _try_read_xpt(_url(folder, base))
        if not ldl_tg.empty:
            break

    # Variable discovery (works across all file types)
    hdl_var = _find_numeric_col(tc_hdl, preferred=["LBDHDD","LBXHDD","LBDHDD_1"], fallback_regex=r"\bHDL\b|HDDS?\b|^LB..HD")
    tc_var  = _find_numeric_col(tc_hdl, preferred=["LBXTC"],                        fallback_regex=r"\bTOTAL.*CHOL|^LB..TC$|\bTC\b")
    # LDL can be in either file; try tc_hdl first, then ldl_tg
    ldl_var = _find_numeric_col(tc_hdl, preferred=["LBDLDL","LBDLDLD","LBDLDLL"],   fallback_regex=r"\bLDL\b|^LB.DLDL")
    if ldl_var is None:
        ldl_var = _find_numeric_col(ldl_tg, preferred=["LBDLDL","LBDLDLD","LBDLDLL"], fallback_regex=r"\bLDL\b|^LB.DLDL")
    tg_var  = _find_numeric_col(ldl_tg, preferred=["LBXTR"],                        fallback_regex=r"\bTRI?G|^LB..TR$")

    # Keep frames
    tc_keep  = tc_hdl[["SEQN", tc_var ]].rename(columns={tc_var : "total_chol_mgdl"}) if tc_var  else pd.DataFrame(columns=["SEQN","total_chol_mgdl"])
    hdl_keep = tc_hdl[["SEQN", hdl_var]].rename(columns={hdl_var: "hdl_mgdl"})       if hdl_var else pd.DataFrame(columns=["SEQN","hdl_mgdl"])
    ldl_keep = (tc_hdl if (ldl_var and ldl_var in tc_hdl.columns) else ldl_tg)
    ldl_keep = ldl_keep[["SEQN", ldl_var]].rename(columns={ldl_var: "ldl_mgdl"})     if ldl_var else pd.DataFrame(columns=["SEQN","ldl_mgdl"])
    tg_keep  = ldl_tg[["SEQN", tg_var]].rename(columns={tg_var: "trig"})             if tg_var  else pd.DataFrame(columns=["SEQN","trig"])

    # --- Build SEQN universe: union of sources ---
    bases = [f for f in (bpq_keep, tc_keep, hdl_keep, ldl_keep, tg_keep) if not f.empty]
    if not bases:
        return pd.DataFrame(columns=["SEQN","period","chol_rx2","hc_dx","hc_lab200","total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"])

    base = bases[0][["SEQN"]].copy()
    for f in bases[1:]:
        base = base.merge(f[["SEQN"]], on="SEQN", how="outer")

    # --- Merge person-level data ---
    df = base
    if not bpq_keep.empty: df = df.merge(bpq_keep, how="left", on="SEQN")
    if not tc_keep.empty:  df = df.merge(tc_keep,  how="left", on="SEQN")
    if not hdl_keep.empty: df = df.merge(hdl_keep, how="left", on="SEQN")
    if not ldl_keep.empty: df = df.merge(ldl_keep, how="left", on="SEQN")
    if not tg_keep.empty:  df = df.merge(tg_keep,  how="left", on="SEQN")

    # Normalize lab dtypes
    for c in ["total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"]:
        if c not in df.columns: df[c] = pd.NA
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("Float64")

    # Friedewald LDL for summaries (not used in SAS flag)
    need_calc = df["ldl_mgdl"].isna()
    if need_calc.any() and df["total_chol_mgdl"].notna().any() and df["hdl_mgdl"].notna().any() and df["trig"].notna().any():
        ldl_est = _friedewald_ldl(df["total_chol_mgdl"], df["hdl_mgdl"], df["trig"])
        df.loc[need_calc, "ldl_mgdl"] = ldl_est.loc[need_calc]

    # --- SAS rule ---
    df["hc_lab200"] = (df["total_chol_mgdl"] > 200).astype("Int64")
    dx = pd.to_numeric(df.get("hc_dx"), errors="coerce").fillna(0).astype(int)
    lab = pd.to_numeric(df.get("hc_lab200"), errors="coerce").fillna(0).astype(int)
    df["chol_rx2"] = ((dx == 1) | (lab == 1)).astype(int)

    df["period"] = label
    keep = ["SEQN","period","chol_rx2","hc_dx","hc_lab200","total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"]
    return df[keep]

# =========================
# Build 1999–2018
# =========================
dfs = []
for suf, lab in CYCLES:
    d = build_cycle_df(suf, lab)
    if not d.empty: dfs.append(d)

df_99_18 = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame(
    columns=["SEQN","period","chol_rx2","hc_dx","hc_lab200","total_chol_mgdl","hdl_mgdl","ldl_mgdl","trig"]
)

# =========================
# Unweighted QC
# =========================
qc_unw = (
    df_99_18.groupby("period", dropna=False)
            .agg(n=("SEQN","count"),
                 chol_rx2_rate=("chol_rx2","mean"),
                 dx_rate=("hc_dx","mean"),
                 lab200_rate=("hc_lab200","mean"),
                 mean_tc=("total_chol_mgdl","mean"),
                 mean_hdl=("hdl_mgdl","mean"),
                 mean_ldl=("ldl_mgdl","mean"))
            .round(3)
            .sort_index()
)
print("\n=== Unweighted QC (SAS-aligned, 1999–2018) ===")
print(qc_unw)

# =========================
# Weighted QC (WTMEC2YR by 2-yr cycle)
# =========================
def fetch_demo_weight_for_cycle(suffix: str):
    folder = _base_folder(suffix)
    d = _try_read_xpt(_url(folder, f"DEMO{suffix}"))
    if d.empty:
        return pd.DataFrame({"SEQN": pd.Series(dtype="Int64"), "W": pd.Series(dtype="float")})
    w = pd.to_numeric(d.get("WTMEC2YR"), errors="coerce")
    out = d[["SEQN"]].copy()
    out["W"] = w if w is not None else 1.0
    return out

rows = []
for suf, lab in CYCLES:
    demo = fetch_demo_weight_for_cycle(suf)
    d = df_99_18[df_99_18["period"] == lab].merge(demo, on="SEQN", how="left")
    rows.append({
        "period": lab,
        "w_union":   round(wmean(d["chol_rx2"],        d["W"]), 3),
        "w_dx":      round(wmean(d["hc_dx"],           d["W"]), 3),
        "w_lab200":  round(wmean(d["hc_lab200"],       d["W"]), 3),
        "w_mean_tc": round(wmean(d["total_chol_mgdl"], d["W"]), 1),
        "w_mean_hdl":round(wmean(d["hdl_mgdl"],        d["W"]), 1),
        "w_mean_ldl":round(wmean(d["ldl_mgdl"],        d["W"]), 1),
    })

qc_w = pd.DataFrame(rows).set_index("period").sort_index()
print("\n=== Weighted QC (WTMEC2YR; SAS-aligned, 1999–2018) ===")
print(qc_w)



=== Unweighted QC (SAS-aligned, 1999–2018) ===
              n  chol_rx2_rate  dx_rate  lab200_rate  mean_tc  mean_hdl  \
period                                                                    
1999-2000  8839          0.333    0.193        0.336  186.956      <NA>   
2001-2002  9682          0.336    0.201        0.324  186.684      <NA>   
2003-2004  8884          0.356    0.243        0.318   185.29    54.472   
2005-2006  8334          0.358    0.226        0.309  184.471      <NA>   
2007-2008  8371          0.417    0.291        0.333  186.675      <NA>   
2009-2010  8763          0.408    0.272        0.326   185.64      <NA>   
2011-2012  8077          0.413    0.323         0.31  183.202      <NA>   
2013-2014  8489          0.400    0.341        0.272  179.534      <NA>   
2015-2016  8285          0.401    0.341        0.279  180.257      <NA>   
2017-2018  7768          0.417    0.358        0.273  179.895      <NA>   

           mean_ldl  
period               
1999-20